In [2]:
import os
import tarfile
import pandas as pd
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from PIL import Image

In [3]:
# Гиперпараметры
EPOCHS = 5
BATCH_SIZE = 8
LR = 1e-3
VAL_SPLIT = 0.2
CLASS_NUM = 102
IMG_SIZE = 250
IMG_CHANNELS = 3
input_shape = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)
DATA_PATH = '../input/flower-dataset-102/'
PATH = "../working/flower/"
RANDOM_SEED = 42

In [4]:
# Функция для извлечения имён файлов
def get_all_filenames(tar_fn):
    with tarfile.open(tar_fn) as f:
        return [m.name for m in f.getmembers() if m.isfile()]


In [5]:
# Чтение данных
df = pd.DataFrame()
df['Id'] = sorted(get_all_filenames(DATA_PATH + "102flowers.tgz"))
df['Category'] = scipy.io.loadmat(DATA_PATH + 'imagelabels.mat')['labels'][0] - 1  # классы (0, 1, 2, ...)
df['Category'] = df['Category'].astype(str)

FileNotFoundError: [Errno 2] No such file or directory: '../input/flower-dataset-102/102flowers.tgz'

In [7]:
pip install kagglehub


  Using cached PyYAML-6.0.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached PyYAML-6.0.2-cp310-cp310-macosx_11_0_arm64.whl (171 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nunenuh/pytorch-challange-flower-dataset")

print("Path to dataset files:", path)

/Users/apple/python/SkillFactory/Ml_learning/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 330M/330M [00:40<00:00, 8.66MB/s] 

Extracting files...


Path to dataset files: /Users/apple/.cache/kagglehub/datasets/nunenuh/pytorch-challange-flower-dataset/versions/3
